# Backtesting with NautilusTrader

---

In this notebook, I'll experiment with a simple backtesting strategy using the **`NautilusTrader`** library. The strategy will be tested on **Bitcoin/USD 1-hour interval data**. This workflow will demonstrate how to set up, implement, and evaluate a trading strategy using historical data.

## Low-level API

The low-level API centers around a `BacktestEngine`, where inputs are initialized and added manually via a Python script. An instantiated `BacktestEngine` can accept the following:

* Lists of `Data` objects, which are automatically sorted into monotonic order based on `ts_init`.
* Multiple venues, manually initialized.
* Multiple actors, manually initialized and added.
* Multiple execution algorithms, manually initialized and added.

This approach offers detailed control over the backtesting process, allowing you to manually configure each component.

In [1]:
import os
import sys
# Add src folder to Python path
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

## Load data

In [2]:
import polars as pl
from src.data.data_pipeline import get_historical_data

In [3]:
df = get_historical_data(download=False)
df.head()

open_time,open,high,low,close,volume,close_time,quote_asset_volume,trades,taker_base_vol,taker_quote_vol,ignore
datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,str
2024-09-24 07:00:00,63305.43,63569.99,63300.0,63559.92,717.69816,2024-09-24 07:59:59.999,4.5550e7,98864,390.39104,2.4774e7,"""0"""
2024-09-24 08:00:00,63559.93,63948.0,63540.0,63851.05,995.90843,2024-09-24 08:59:59.999,6.3463e7,131479,496.91258,3.1667e7,"""0"""
2024-09-24 09:00:00,63851.05,63893.51,63484.0,63524.5,729.48652,2024-09-24 09:59:59.999,4.6436e7,123317,292.08143,1.8592e7,"""0"""
2024-09-24 10:00:00,63524.5,63883.0,63504.1,63526.0,618.45246,2024-09-24 10:59:59.999,3.9361e7,101884,370.79941,2.3604e7,"""0"""
2024-09-24 11:00:00,63526.01,63600.0,63380.0,63478.34,591.1906,2024-09-24 11:59:59.999,3.7530e7,92450,260.37361,1.6529e7,"""0"""


### POC design — Data ingest

In [4]:
from __future__ import annotations
import polars as pl
from pathlib import Path
from nautilus_trader.model import BarType
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from decimal import Decimal

In [5]:
CATALOG_PATH = Path.cwd() / "catalog"
CATALOG_PATH.mkdir(parents=True, exist_ok=True)

# Create a catalog instance
catalog = ParquetDataCatalog(CATALOG_PATH)

Create instrument with nt fast example

In [6]:
BTCUSD = TestInstrumentProvider.btcusdt_binance()
print(BTCUSD)

CurrencyPair(id=BTCUSDT.BINANCE, raw_symbol=BTCUSDT, asset_class=CRYPTOCURRENCY, instrument_class=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)


or do it manually

In [7]:
from nautilus_trader.model.currencies import BTC
from nautilus_trader.model.currencies import USDT
from nautilus_trader.model.instruments import CurrencyPair
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.identifiers import Symbol
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.objects import Money
from nautilus_trader.model.objects import Price
from nautilus_trader.model.objects import Quantity

BTCUSD = CurrencyPair(
    instrument_id=InstrumentId(                 # Unique instrument identifier combining symbol and venue
        symbol=Symbol("BTCUSDT"),               # Exchange symbol string
        venue=Venue("BINANCE"),                 # Trading venue identifier
    ),
    raw_symbol=Symbol("BTCUSDT"),               # Raw exchange symbol as listed
    base_currency=BTC,                          # Base currency in the pair (asset being traded)
    quote_currency=USDT,                        # Quote currency (asset used to price the base)
    price_precision=2,                          # Decimal places allowed in prices
    size_precision=6,                           # Decimal places allowed in order size
    price_increment=Price(1e-02, precision=2),  # Minimum tick size for price changes
    size_increment=Quantity(1e-06, precision=6),# Minimum step size for order quantity
    lot_size=None,                              # Minimum tradable lot size (None means unrestricted)
    max_quantity=Quantity(9000, precision=6),   # Maximum allowable order quantity
    min_quantity=Quantity(1e-06, precision=6),  # Minimum allowable order quantity
    max_notional=None,                          # Maximum notional order value (None = not enforced)
    min_notional=Money(10.00000000, USDT),      # Minimum notional order value
    max_price=Price(1000000, precision=2),      # Maximum allowable order price
    min_price=Price(0.01, precision=2),         # Minimum allowable order price
    margin_init=Decimal(0),                     # Initial margin requirement (for leveraged instruments)
    margin_maint=Decimal(0),                    # Maintenance margin requirement
    maker_fee=Decimal("0.001"),                 # Maker fee rate
    taker_fee=Decimal("0.001"),                 # Taker fee rate
    ts_event=0,                                 # Timestamp of event creation (epoch ns)
    ts_init=0,                                  # Timestamp of initialization (epoch ns)
)
print(BTCUSD)

CurrencyPair(id=BTCUSDT.BINANCE, raw_symbol=BTCUSDT, asset_class=CRYPTOCURRENCY, instrument_class=SPOT, quote_currency=USDT, is_inverse=False, price_precision=2, price_increment=0.01, size_precision=6, size_increment=0.000001, multiplier=1, lot_size=None, margin_init=0, margin_maint=0, maker_fee=0.001, taker_fee=0.001, info=None)


OBLIGATORIO: Pandas df con timestamp como indice

In [8]:
df = df.with_columns(
        (pl.col("open_time") + pl.duration(hours=1)).alias("timestamp"),
    ).to_pandas()

# Change order of columns
df = df.reindex(columns=["timestamp", "open", "high", "low", "close", "volume"])
df = df.set_index("timestamp")
df.head()

,open,high,low,close,volume
timestamp,,,,,
2024-09-24 08:00:00,63305.43,63569.99,63300.0,63559.92,717.69816
2024-09-24 09:00:00,63559.93,63948.00,63540.0,63851.05,995.90843
2024-09-24 10:00:00,63851.05,63893.51,63484.0,63524.50,729.48652
2024-09-24 11:00:00,63524.50,63883.00,63504.1,63526.00,618.45246
2024-09-24 12:00:00,63526.01,63600.00,63380.0,63478.34,591.19060


In [9]:
EURUSD_SPOT_1HOUR_BARTYPE = BarType.from_str(
    f"{BTCUSD.id}-1-HOUR-LAST-EXTERNAL"
)

In [10]:
wrangler = BarDataWrangler(bar_type=EURUSD_SPOT_1HOUR_BARTYPE, instrument=BTCUSD)
BARS = wrangler.process(df)

## Simple bt

In [11]:
from nautilus_trader.model.currencies import BTC
from nautilus_trader.model.currencies import USDT

Configure Engine:

The BacktestDataConfig objects are integrated into the backtesting framework through BacktestRunConfig:
(https://nautilustrader.io/docs/latest/concepts/data/#integration-with-backtestrunconfig)

In [12]:
from nautilus_trader.adapters.binance import BINANCE_VENUE
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.identifiers import TraderId
from nautilus_trader.model.objects import Money

# Configure backtest engine
engine_config = BacktestEngineConfig(
    trader_id=TraderId("BACKTESTER-ENGINE-001"),
    logging=LoggingConfig(log_level="INFO"),
)

# Build the backtest engine
engine = BacktestEngine(config=engine_config)

# Add Venue
instrument = BTCUSD

engine.add_venue(
    venue=BINANCE_VENUE,
    oms_type=OmsType.NETTING,
    account_type=AccountType.CASH,
    base_currency=None,
    starting_balances=[Money(1_000_000.0, USDT), Money(1.0, BTC)],
)

2025-10-13T07:11:37.208705000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: =================================================================
2025-10-13T07:11:37.208720000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-10-13T07:11:37.208720001Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-10-13T07:11:37.208720002Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-10-13T07:11:37.208721000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: =================================================================
2025-10-13T07:11:37.208721001Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: 
2025-10-13T07:11:37.208721002Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-10-13T07:11:37.208721003Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-10-13T07:11:37.208721004Z [INFO] BACKTESTER-ENGINE-001.B

In [13]:
engine.add_instrument(instrument)

bar_type = EURUSD_SPOT_1HOUR_BARTYPE

engine.add_data(BARS)

2025-10-13T07:11:37.225192000Z [INFO] BACKTESTER-ENGINE-001.SimulatedExchange(BINANCE): Added instrument BTCUSDT.BINANCE and created matching engine
2025-10-13T07:11:37.225197000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: Added BTCUSDT.BINANCE Instrument
2025-10-13T07:11:37.228713000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: Added 8_760 BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL Bar elements


In [14]:
from src.strategies.nautilus import SmaCrossConfig, SmaCrossNT

strategy_config = SmaCrossConfig(
    strategy_id="SMA-CROSS-LONG-001",
    instrument_id=BTCUSD.id,
    bar_type=EURUSD_SPOT_1HOUR_BARTYPE,
    trade_size=Decimal("1"),
    fast_sma_period = 30,
    slow_sma_period = 100
    )

# Instantiate and add your strategy
strategy = SmaCrossNT(strategy_config)
engine.add_strategy(strategy=strategy)

2025-10-13T07:11:37.233300000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: READY
2025-10-13T07:11:37.233354000Z [INFO] BACKTESTER-ENGINE-001.ExecEngine: Registered OMS.UNSPECIFIED for Strategy SMA-000
2025-10-13T07:11:37.233358000Z [INFO] BACKTESTER-ENGINE-001.BACKTESTER-ENGINE-001: Registered Strategy SMA-000


In [ ]:
# Run the engine (from start to end of data)
engine.run()

2025-10-13T07:11:37.236020000Z [INFO] BACKTESTER-ENGINE-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=CASH, base_currency=None, is_reported=True, balances=[AccountBalance(total=1_000_000.00000000 USDT, locked=0.00000000 USDT, free=1_000_000.00000000 USDT), AccountBalance(total=1.00000000 BTC, locked=0.00000000 BTC, free=1.00000000 BTC)], margins=[], event_id=915d8e0a-8e78-4a94-a55b-9cfec76470e9)
2024-09-24T08:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.BacktestEngine: STARTING
2024-09-24T08:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: Connecting...
2024-09-24T08:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: Connected
2024-09-24T08:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: RUNNING
2024-09-24T08:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.DataEngine: RUNNING
2024-09-24T08:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.RiskEngine: RUNNING
2024-09-24T08:00:00.000000000Z [INFO] BACKTESTER-ENGINE-0

SDT.BINANCE-1-HOUR-LAST-EXTERNAL,66668.99,66683.34,66362.15,66414.01,372.218790,1729720800000000000)
2024-10-23T23:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,66414.00,66575.76,66351.58,66472.96,462.358380,1729724400000000000)
2024-10-24T00:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,66472.95,66807.37,66460.00,66668.65,589.808940,1729728000000000000)
2024-10-24T01:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,66668.65,67476.11,66510.00,67432.01,1333.572780,1729731600000000000)
2024-10-24T02:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,67432.00,67612.12,67058.03,67268.17,1439.994360,1729735200000000000)
2024-10-24T03:00:00.000000000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: Bar(BTCUSDT.BINANCE-1-HOUR-LAST-EXTERNAL,67268.17,67507.60,67

In [16]:
account_report = pl.DataFrame(engine.trader.generate_account_report(BINANCE_VENUE))
orders_report = pl.DataFrame(engine.trader.generate_order_fills_report())
positions_report = pl.DataFrame(engine.trader.generate_positions_report())

In [17]:
account_report

total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
str,str,str,str,str,str,str,list[null],bool,struct[0]
"""1000000.00000000""","""0.00000000""","""1000000.00000000""","""USDT""","""BINANCE-001""","""CASH""",null,[],true,{}
"""1.00000000""","""0.00000000""","""1.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],true,{}
"""934170.22599000""","""0.00000000""","""934170.22599000""","""USDT""","""BINANCE-001""","""CASH""",null,[],false,{}
"""2.00000000""","""0.00000000""","""2.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],false,{}
"""998417.49441000""","""0.00000000""","""998417.49441000""","""USDT""","""BINANCE-001""","""CASH""",null,[],false,{}
…,…,…,…,…,…,…,…,…,…
"""1.00000000""","""0.00000000""","""1.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],false,{}
"""894715.99868000""","""0.00000000""","""894715.99868000""","""USDT""","""BINANCE-001""","""CASH""",null,[],false,{}
"""2.00000000""","""0.00000000""","""2.00000000""","""BTC""","""BINANCE-001""","""CASH""",null,[],false,{}


In [18]:
orders_report

trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,time_in_force,is_reduce_only,is_quote_quantity,filled_qty,liquidity_side,avg_px,slippage,commissions,emulation_trigger,status,contingency_type,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
str,str,str,str,str,str,str,str,str,str,str,bool,bool,str,str,f64,f64,list[str],str,str,str,str,str,str,str,str,str,str,str,"datetime[ns, UTC]","datetime[ns, UTC]"
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-001""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-101""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",65764.01,0.0,"[""65.76401000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""3f6eb532-799e-4d47-9b23-450071…",2024-09-28 11:00:00 UTC,2024-09-28 11:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-002""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-148""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",64311.58,0.0,"[""64.31158000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""11d67144-b829-4087-82b4-3772e9…",2024-09-30 09:00:00 UTC,2024-09-30 09:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-003""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-266""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",62149.47,0.0,"[""62.14947000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""7ce0c057-074c-4254-ad6b-edc230…",2024-10-05 06:00:00 UTC,2024-10-05 06:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-004""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-359""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",62414.55,0.0,"[""62.41455000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""dd410d1f-9378-4b51-8662-d0cc51…",2024-10-09 02:00:00 UTC,2024-10-09 02:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-005""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-435""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",62583.58,0.0,"[""62.58358000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""8a2d2765-9d70-4c47-a888-13cb9b…",2024-10-12 05:00:00 UTC,2024-10-12 05:00:00 UTC
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-104""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-8441""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",110384.15,0.0,"[""110.38415000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""8b129dcd-d79a-4dc0-a979-705311…",2025-09-06 16:00:00 UTC,2025-09-06 16:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-105""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-8479""","""MARKET""","""BUY""","""1.000000""","""IOC""",false,false,"""1.000000""","""TAKER""",110979.35,0.0,"[""110.97935000 USDT""]","""NO_TRIGGER""","""FILLED""","""NO_CONTINGENCY""",null,null,null,null,null,null,null,"""7a04a772-e1d1-43df-abda-58f811…",2025-09-08 05:00:00 UTC,2025-09-08 05:00:00 UTC
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-1-106""","""BTCUSDT.BINANCE-SMA-000""","""BINANCE-001""","""BINANCE-1-8658""","""MARKET""","""SELL""","""1.000000""","""GTC""",true,false,"""1.000000""","""TAKER""",114679.56,0.0,"[""1

In [19]:
positions_report

trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl,is_snapshot
str,str,str,str,str,str,str,str,str,str,i64,"datetime[ns, UTC]",i64,"datetime[ns, UTC]",i64,f64,f64,list[str],f64,str,bool
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20240928-110000-001-000-1""","""O-20240930-090000-001-000-2""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1727521200000000000,2024-09-28 11:00:00 UTC,1727686800000000000,2024-09-30 09:00:00 UTC,165600000000000,65764.01,64311.58,"[""130.07559000 USDT""]",-0.02209,"""-1582.50559000 USDT""",true
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20241005-060000-001-000-3""","""O-20241009-020000-001-000-4""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1728108000000000000,2024-10-05 06:00:00 UTC,1728439200000000000,2024-10-09 02:00:00 UTC,331200000000000,62149.47,62414.55,"[""124.56402000 USDT""]",0.00427,"""140.51598000 USDT""",true
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20241012-050000-001-000-5""","""O-20241022-020000-001-000-6""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1728709200000000000,2024-10-12 05:00:00 UTC,1729562400000000000,2024-10-22 02:00:00 UTC,853200000000000,62583.58,67508.29,"[""130.09187000 USDT""]",0.07869,"""4794.61813000 USDT""",true
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20241025-020000-001-000-7""","""O-20241026-150000-001-000-8""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1729821600000000000,2024-10-25 02:00:00 UTC,1729954800000000000,2024-10-26 15:00:00 UTC,133200000000000,67969.7,66797.99,"[""134.76769000 USDT""]",-0.01724,"""-1306.47769000 USDT""",true
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20241027-230000-001-000-9""","""O-20241101-080000-001-000-10""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1730070000000000000,2024-10-27 23:00:00 UTC,1730448000000000000,2024-11-01 08:00:00 UTC,378000000000000,68124.0,69273.7,"[""137.39770000 USDT""]",0.01688,"""1012.30230000 USDT""",true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20250823-010000-001-000-99""","""O-20250825-040000-001-000-100""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1755910800000000000,2025-08-23 01:00:00 UTC,1756094400000000000,2025-08-25 04:00:00 UTC,183600000000000,116870.22,112916.84,"[""229.78706000 USDT""]",-0.03383,"""-4183.16706000 USDT""",true
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20250828-110000-001-000-101""","""O-20250829-180000-001-000-102""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1756378800000000000,2025-08-28 11:00:00 UTC,1756490400000000000,2025-08-29 18:00:00 UTC,111600000000000,113127.53,108466.01,"[""221.59354000 USDT""]",-0.04121,"""-4883.11354000 USDT""",true
"""BACKTESTER-ENGINE-001""","""SMA-000""","""BTCUSDT.BINANCE""","""BINANCE-001""","""O-20250902-070000-001-000-103""","""O-20250906-160000-001-000-104""","""BUY""","""FLAT""","""0.000000""","""1.000000""",1756796400000000000,2025-09-02 07:00:00 UTC,1757174400000000000,2025-09-06 16:00:00 UTC,378000000000000,110376.63,110384.15,"[""220.76078000 USDT""]",0.00007,"""-213.24078000 USDT""",true


In [20]:
# For repeated backtest runs make sure to reset the engine
engine.reset()

# Good practice to dispose of the object
engine.dispose()

2025-10-13T07:11:41.191226000Z [INFO] BACKTESTER-ENGINE-001.DataClient-BINANCE: READY
2025-10-13T07:11:41.191240000Z [INFO] BACKTESTER-ENGINE-001.DataEngine: READY
2025-10-13T07:11:41.191251000Z [INFO] BACKTESTER-ENGINE-001.ExecClient-BINANCE: READY
2025-10-13T07:11:41.191421000Z [INFO] BACKTESTER-ENGINE-001.Cache: Reset
2025-10-13T07:11:41.191427000Z [INFO] BACKTESTER-ENGINE-001.ExecEngine: READY
2025-10-13T07:11:41.191436000Z [INFO] BACKTESTER-ENGINE-001.RiskEngine: READY
2025-10-13T07:11:41.191449000Z [INFO] BACKTESTER-ENGINE-001.OrderEmulator: READY
2025-10-13T07:11:41.191498000Z [INFO] BACKTESTER-ENGINE-001.SMA-CROSS-LONG-001: READY
2025-10-13T07:11:41.191604000Z [INFO] BACKTESTER-ENGINE-001.Portfolio: READY
2025-10-13T07:11:41.191610000Z [INFO] BACKTESTER-ENGINE-001.BACKTESTER-ENGINE-001: READY
2025-10-13T07:11:41.191682000Z [INFO] BACKTESTER-ENGINE-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=CASH, base_currency=None, is_reported=True, balances=[Accou